To only run one time at the beginning of the test (unless you want to change the current overlay) :

In [1]:
from pynq import Overlay
ol = Overlay("/home/xilinx/pynq/overlays/qspi/design_qspi.bit")

To run only if you made change in the overlay :

In [2]:
ol.download()
# ol?

Real test code begins : 

In [3]:
from pynq import MMIO

SPI_regSpace = ol.ip_dict['axi_quad_spi_0']
IP_BASE_ADDRESS = SPI_regSpace['phys_addr']
IP_ADDRESS_RNGE = SPI_regSpace['addr_range']
AxiQspi = MMIO(IP_BASE_ADDRESS, IP_ADDRESS_RNGE)

In [4]:
XSP_DGIER_OFFSET  = 0x1C
XSP_IISR_OFFSET   = 0x20
XSP_IIER_OFFSET   = 0x28
XSP_SRR_OFFSET    = 0x40  # Software reset register
XSP_CR_OFFSET     = 0x60  # SPI control register
XSP_SR_OFFSET     = 0x64  # SPI status register
XSP_DTR_OFFSET    = 0x68  # SPI data transmit register. A singleregister or a FIFO
XSP_DRR_OFFSET    = 0x6C  # SPI data receive register. A singleregister or a FIFO
XSP_SSR_OFFSET    = 0x70  # SPI Slave select register
XSP_TFO_OFFSET    = 0x74  # Transmit FIFO occupancy register
XSP_RFO_OFFSET    = 0x78  # Receive FIFO occupancy register
XSP_REGISTERS     = [0x40, 0x60, 0x64, 0x68, 0x6c, 0x70, 0x74, 0x78, 0x1c, 0x20, 0x28]

XSP_SRR_RESET_MASK         = 0x0A
XSP_SR_TX_EMPTY_MASK       = 0x00000004
XSP_SR_TX_FULL_MASK        = 0x00000008
XSP_CR_TRANS_INHIBIT_MASK  = 0x00000100
XSP_CR_LOOPBACK_MASK       = 0x00000001
XSP_CR_ENABLE_MASK         = 0x00000002
XSP_CR_MASTER_MODE_MASK    = 0x00000004
XSP_CR_CLK_POLARITY_MASK   = 0x00000008
XSP_CR_CLK_PHASE_MASK      = 0x00000010
XSP_CR_TXFIFO_RESET_MASK   = 0x00000020
XSP_CR_RXFIFO_RESET_MASK   = 0x00000040
XSP_CR_MANUAL_SS_MASK      = 0x00000080

SLAVE_NO_SELECTION = 0xFFFFFFFF

def cnfg(AxiQspi, clk_phase=0, clk_pol=0):
    print("Configure device")
    # Reset the SPI device
    AxiQspi.write(XSP_SRR_OFFSET, XSP_SRR_RESET_MASK)
    # Enable the transmit empty interrupt, which we use to determine progress on the transmission. 
    AxiQspi.write(XSP_IIER_OFFSET, XSP_SR_TX_EMPTY_MASK)
    # Disable the global IPIF interrupt
    AxiQspi.write(XSP_DGIER_OFFSET, 0)
    # Deselect the slave on the SPI bus
    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
    # Disable the transmitter, enable Manual Slave Select Assertion, put SPI controller into master mode, and enable it
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK | XSP_CR_TXFIFO_RESET_MASK | XSP_CR_RXFIFO_RESET_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg & ~(XSP_CR_CLK_PHASE_MASK | XSP_CR_CLK_POLARITY_MASK) 
    if clk_phase == 1:
        ControlReg = ControlReg | XSP_CR_CLK_PHASE_MASK
    if clk_pol == 1:
        ControlReg = ControlReg | XSP_CR_CLK_POLARITY_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    return 0

def xfer(packet, AxiQspi):
    print("TransferData")
    for data in packet:
        AxiQspi.write(XSP_SSR_OFFSET, 0xFFFFFFFE)
        AxiQspi.write(XSP_DTR_OFFSET, data)
        ControlReg = AxiQspi.read(XSP_CR_OFFSET)
        ControlReg = ControlReg & ~XSP_CR_TRANS_INHIBIT_MASK
        AxiQspi.write(XSP_CR_OFFSET, ControlReg)

        StatusReg = AxiQspi.read(XSP_SR_OFFSET)
        while (StatusReg & XSP_SR_TX_EMPTY_MASK) == 0:
            StatusReg = AxiQspi.read(XSP_SR_OFFSET)

        print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(AxiQspi.read(XSP_RFO_OFFSET)))
        ControlReg = AxiQspi.read(XSP_CR_OFFSET)
        ControlReg = ControlReg | XSP_CR_TRANS_INHIBIT_MASK
        AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)

    print("ReadResponse")
    resp = list()
    RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01
    while RxFifoStatus == 0:
        temp = AxiQspi.read(XSP_RFO_OFFSET)
        print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(temp))
        temp = AxiQspi.read(XSP_DRR_OFFSET)
        print('XSP_DRR_OFFSET  : 0x{0:08x}'.format(temp))    
        resp.append(temp)
        RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01

    return resp

In [5]:
from pynq import Clocks

print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')

FCLK0: 50.000000MHz


In [6]:
cnfg(AxiQspi,0,0)

Configure device


0

In [9]:
cnfg(AxiQspi,0,0)
resp = xfer([0xDA,0x0E,0xDB,0xDA,0x00], AxiQspi)

Configure device
TransferData
XSP_RFO_OFFSET  : 0x00000000
XSP_RFO_OFFSET  : 0x00000001
XSP_RFO_OFFSET  : 0x00000002
XSP_RFO_OFFSET  : 0x00000003
XSP_RFO_OFFSET  : 0x00000004
ReadResponse
XSP_RFO_OFFSET  : 0x00000004
XSP_DRR_OFFSET  : 0x00000000
XSP_RFO_OFFSET  : 0x00000003
XSP_DRR_OFFSET  : 0x000000da
XSP_RFO_OFFSET  : 0x00000002
XSP_DRR_OFFSET  : 0x0000000e
XSP_RFO_OFFSET  : 0x00000001
XSP_DRR_OFFSET  : 0x000000db
XSP_RFO_OFFSET  : 0x00000000
XSP_DRR_OFFSET  : 0x000000da


In [8]:
print(resp)
for data in resp :
    print('XSP_DRR_OFFSET  : 0x{0:02x}'.format(data))   

[0, 218, 14, 219, 218]
XSP_DRR_OFFSET  : 0x00
XSP_DRR_OFFSET  : 0xda
XSP_DRR_OFFSET  : 0x0e
XSP_DRR_OFFSET  : 0xdb
XSP_DRR_OFFSET  : 0xda


In [17]:
resp = xfer([0x11, 12, 0x0], AxiQspi)

TransferData
XSP_RFO_OFFSET  : 0x00000000
XSP_RFO_OFFSET  : 0x00000001
XSP_RFO_OFFSET  : 0x00000002
ReadResponse
XSP_RFO_OFFSET  : 0x00000002
XSP_DRR_OFFSET  : 0x00000000
XSP_RFO_OFFSET  : 0x00000001
XSP_DRR_OFFSET  : 0x00000011
XSP_RFO_OFFSET  : 0x00000000
XSP_DRR_OFFSET  : 0x0000000c
